# **ABCDEats, Inc. Dashboard**

# 1. Imports and Loading

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import itertools
import joblib

from dash import Dash, html, dash_table, dcc, callback, Output, Input, MATCH, callback_context, State, dash
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from plotly.subplots import make_subplots
import plotly.colors as colors

from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_score

In [2]:
# Ensuring pandas always prints all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 1000)

In [3]:
# Load the data
PATH = 'data/'

In [4]:
data = pd.read_csv(f'{PATH}raw_data.csv', index_col=0)
regulars = pd.read_csv(f'{PATH}regulars.csv', index_col=0)

# 2. Mapping

In [5]:
mapping_dict = {
    'cust_region': 'Region',
    'cust_age': 'Age',
    'n_vendor': 'Vendor Count',
    'n_product': 'Product Count',
    'n_chain': 'Chain Restaurant Order Count',
    'first_order': 'First Order Date',
    'last_order': 'Last Order Date',
    'last_promo': 'Promotion',
    'pay_method': 'Payment Method',
    'american': 'American',
    'asian': 'Asian',
    'beverages': 'Beverages',
    'cafe': 'Cafe',
    'chicken_dishes': 'Chicken Dishes',
    'chinese': 'Chinese',
    'desserts': 'Desserts',
    'healthy': 'Healthy',
    'indian': 'Indian',
    'italian': 'Italian',
    'japanese': 'Japanese',
    'noodle_dishes': 'Noodle Dishes',
    'other': 'Other Cuisines',
    'street_food_snacks': 'Street Food & Snacks',
    'thai': 'Thai',
    'DOW_0': 'Sunday',
    'DOW_1': 'Monday',
    'DOW_2': 'Tuesday',
    'DOW_3': 'Wednesday',
    'DOW_4': 'Thursday',
    'DOW_5': 'Friday',
    'DOW_6': 'Saturday',
    'HR_0': '12AM',
    'HR_1': '1AM',
    'HR_2': '2AM',
    'HR_3': '3AM',
    'HR_4': '4AM',
    'HR_5': '5AM',
    'HR_6': '6AM',
    'HR_7': '7AM',
    'HR_8': '8AM',
    'HR_9': '9AM',
    'HR_10': '10AM',
    'HR_11': '11AM',
    'HR_12': '12PM',
    'HR_13': '1PM',
    'HR_14': '2PM',
    'HR_15': '3PM',
    'HR_16': '4PM',
    'HR_17': '5PM',
    'HR_18': '6PM',
    'HR_19': '7PM',
    'HR_20': '8PM',
    'HR_21': '9PM',
    'HR_22': '10PM',
    'HR_23': '11PM',
    'cust_city': 'City',
    'total_amt': 'Total Amount',
    'n_order': 'Order Count',
    'avg_amt_per_product': 'Average Amount per Product',
    'avg_amt_per_order': 'Average Amount per Order',
    'avg_amt_per_vendor': 'Average Amount per Vendor',
    'days_cust': 'Days as Customer',
    'avg_days_to_order': 'Average Days to Order',
    'days_due': 'Order Days Due',
    'per_chain_order': 'Percentage of Orders in Chain Restaurant',
    'n_days_week': 'Days of Week Ordered Count',
    'n_times_day': 'Hours Ordered Count',
    'regular': 'Is Regular',
    'n_cuisines': 'Cuisines Count',
    'log_n_vendor': 'Log Vendor Count',
    'log_n_product': 'Log Product Count',
    'log_n_chain': 'Log Chain Restaurant Order Count',
    'log_american': 'Log American',
    'log_asian': 'Log Asian',
    'log_beverages': 'Log Beverages',
    'log_cafe': 'Log Cafe',
    'log_chicken_dishes': 'Log Chicken Dishes',
    'log_chinese': 'Log Chinese',
    'log_desserts': 'Log Desserts',
    'log_healthy': 'Log Healthy',
    'log_indian': 'Log Indian',
    'log_italian': 'Log Italian',
    'log_japanese': 'Log Japanese',
    'log_noodle_dishes': 'Log Noodle Dishes',
    'log_other': 'Log Other Cuisines',
    'log_street_food_snacks': 'Log Street Food & Snacks',
    'log_thai': 'Log Thai',
    'log_total_amt': 'Log Total Amount',
    'log_n_order': 'Log Order Count',
    'log_avg_amt_per_product': 'Log Average Amount per Product',
    'log_avg_amt_per_order': 'Log Average Amount per Order',
    'log_avg_amt_per_vendor': 'Log Average Amount per Vendor',
    'log_n_days_week': 'Log Days of Week Ordered Count',
    'log_n_times_day': 'Log Hours Ordered Count',
    'foodie_flag': 'Is Foodie',
    'gluttonous_flag': 'Is Gluttonous',
    'loyal_flag': 'Is Loyal',
    'top_cuisine': 'Top Cuisine',
    'avg_amt_per_day': 'Average Amount Spent per Day',
    'avg_product_per_day': 'Average Products Ordered per Day',
    'avg_order_per_day': 'Average Orders Placed per Day',
    'age_bucket': 'Age Bucket',
    'transaction_volume': 'Transaction Volume',
    'interaction_rate': 'Interaction Rate'
}

In [6]:
data.rename(columns=mapping_dict, inplace=True)

# 2. Clustering

In [7]:
encoded_features = [
    *[col for col in regulars.columns if col.startswith('cust_region')],
    *[col for col in regulars.columns if col.startswith('last_promo')],
    *[col for col in regulars.columns if col.startswith('pay_method')],
    *[col for col in regulars.columns if col.startswith('cust_city')],
    *[col for col in regulars.columns if col.startswith('age_bucket')]
]

In [8]:
encoder = joblib.load('models/hot_encoder.pkl')

regulars = pd.concat([
    regulars[[col for col in regulars.columns if col not in encoded_features]],
    pd.DataFrame(
        encoder.inverse_transform(regulars[encoded_features]),
        columns=['cust_region', 'last_promo', 'pay_method', 'cust_city', 'age_bucket'],
        index=regulars.index
    )    
], axis=1)

## 2.1. Spending Diversity

In [9]:
spending_diversity_features = ['total_amt', 'n_cuisines', 'n_vendor', 'n_product']
spending_diversity_df = regulars[spending_diversity_features].copy()

spending_diversity_algorithm = joblib.load('models/spending_clustering.pkl')

spending_labels = spending_diversity_algorithm.predict(spending_diversity_df)

spending_diversity_df = pd.concat([
    spending_diversity_df,
    pd.Series(spending_labels, name='labels', index=spending_diversity_df.index),
    regulars[[col for col in regulars.columns if col not in spending_diversity_features]]
], axis=1)

In [10]:
spending_diversity_profiling = pd.concat([
    spending_diversity_df['labels'],
    encoder.transform(regulars[['cust_region', 'last_promo', 'pay_method', 'cust_city', 'age_bucket']])
], axis=1)

In [11]:
std_scaler = joblib.load('models/std_scaler.pkl')

std_scaler_features = [
    'cust_age', 'n_vendor', 'n_product', 'n_chain', 'first_order', 'last_order', 'american', 'asian', 'beverages', 'cafe',
    'chicken_dishes', 'chinese', 'desserts', 'healthy', 'indian', 'italian', 'japanese', 'noodle_dishes', 'other', 'street_food_snacks', 'thai',
    'DOW_0', 'DOW_1', 'DOW_2', 'DOW_3', 'DOW_4', 'DOW_5', 'DOW_6', 'HR_0', 'HR_1', 'HR_2', 'HR_3', 'HR_4', 'HR_5', 'HR_6', 'HR_7', 'HR_8', 'HR_9',
    'HR_10', 'HR_11', 'HR_12', 'HR_13', 'HR_14', 'HR_15', 'HR_16', 'HR_17', 'HR_18', 'HR_19', 'HR_20', 'HR_21', 'HR_22', 'HR_23', 'total_amt',
    'n_order', 'avg_amt_per_product', 'avg_amt_per_order', 'avg_amt_per_vendor', 'days_cust', 'avg_days_to_order', 'days_due',
    'per_chain_order', 'n_days_week', 'n_times_day', 'n_cuisines', 'log_n_vendor', 'log_n_product', 'log_n_chain', 'log_american',
    'log_asian', 'log_beverages', 'log_cafe', 'log_chicken_dishes', 'log_chinese', 'log_desserts', 'log_healthy', 'log_indian',
    'log_italian', 'log_japanese', 'log_noodle_dishes', 'log_other', 'log_street_food_snacks', 'log_thai', 'log_total_amt', 'log_n_order',
    'log_avg_amt_per_product', 'log_avg_amt_per_order', 'log_avg_amt_per_vendor', 'log_n_days_week', 'log_n_times_day',
    'avg_amt_per_day', 'avg_product_per_day', 'avg_order_per_day'
]

spending_diversity_df = pd.concat([
    pd.DataFrame(std_scaler.inverse_transform(spending_diversity_df[std_scaler_features]), columns=std_scaler_features, index=spending_diversity_df.index),
    spending_diversity_df.drop(columns=std_scaler_features)
], axis=1)

In [12]:
spending_diversity_df.drop(columns=[col for col in spending_diversity_df.columns if col.startswith('log_')], inplace=True)

In [13]:
spending_diversity_df.rename(columns=mapping_dict, inplace=True)
spending_diversity_features = [mapping_dict.get(val, val) for val in spending_diversity_features]

## 2.2. Geography

In [14]:
# geography_features = []
# geography_df = regulars[geography_features].copy()

# geography_algorithm = joblib.load('models/city_clustering.pkl')

# geography_labels = geography_algorithm.predict(geography_df)

# geography_df = pd.concat([
#     geography_df,
#     pd.Series(geography_labels, name='labels', index=geography_df.index),
#     regulars[[col for col in regulars.columns if col not in geography_features]]
# ], axis=1)

## 2.3. Cuisines

In [15]:
# cuisines_features = []
# cuisines_df = regulars[cuisines_features].copy()

# cuisines_algorithm = joblib.load('models/cuisine_clustering.pkl')

# cuisines_labels = cuisines_algorithm.predict(cuisines_df)

# cuisines_df = pd.concat([
#     cuisines_df,
#     pd.Series(cuisines_labels, name='labels', index=cuisines_df.index),
#     regulars[[col for col in regulars.columns if col not in cuisines_features]]
# ], axis=1)

## 2.4. Time

In [16]:
# time_features = []
# time_df = regulars[time_features].copy()

# time_algorithm = joblib.load('models/city_clustering.pkl')

# time_labels = time_algorithm.predict(time_df)

# time_df = pd.concat([
#     time_df,
#     pd.Series(time_labels, name='labels', index=time_df.index),
#     regulars[[col for col in regulars.columns if col not in time_features]]
# ], axis=1)

# 4. Building the Dashboard

In [75]:
non_metric_features = [
    'Region', 'Promotion', 'Payment Method', 'City', 'Age Bucket', 
    'Is Regular', 'Is Foodie', 'Is Gluttonous', 'Is Loyal', 'Top Cuisine'
]
no_categorical = [col for col in data.columns if col not in non_metric_features]

In [76]:
# Initialize the app
external_stylesheet = [dbc.themes.VAPOR]
app = Dash(__name__, external_stylesheets=external_stylesheet, suppress_callback_exceptions=True)

In [77]:
# App layout
app.layout = html.Div([
    dbc.Navbar(
        dbc.Container([
            dbc.Nav([
                dbc.NavItem(dbc.NavLink("Home", href="/", id='home-button'), style={'margin-right': '15px'}),
                dbc.NavItem(dbc.NavLink("Basic Exploration", href="/single-feature", id='single-feature-button'), style={'margin-right': '15px'}),
                dbc.NavItem(dbc.NavLink("Pairplot Exploration", href="/pairplot", id='pairplot-button'), style={'margin-right': '15px'}),
                dbc.DropdownMenu([
                    dbc.DropdownMenuItem("Spending Diversity", href="/spending", id='spending-diversity-button'),
                    dbc.DropdownMenuItem("Geography", href="", id='geography-button'),
                    dbc.DropdownMenuItem("Cuisines", href="", id='cuisines-button'),
                    dbc.DropdownMenuItem("Time", href="", id='time-button'),
                ], label='Clustering', nav=True, in_navbar=True)
            ], className='mr-aulo', pills=True),
            dbc.NavbarBrand("ABCDEats, Inc.", href="#", className='ml-aulo')
        ]), color='primary', dark=True
    ),
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

In [78]:
home_layout = html.Div([
    dbc.Container([
        dbc.Row([dbc.Col(html.H1("This is a project developed by Martins & Fonseca Consulting on behalf of the Data Mining course"), width=12)])
    ])
])

## Single Feature

In [79]:
single_feature = html.Div([
    dbc.Container([
        dbc.Row([
            html.Div('Basic Exploration', className="text-primary text-center fs-3", style={'margin-bottom': '15px'})
        ]),
        dbc.Row([
            dbc.Col([
                html.Label("Select a Feature to Display:", style={'margin-bottom': '10px'}),
                dbc.Select(
                    id='column-dropdown',
                    options=[{'label': col, 'value': col} for col in data.columns],
                    value=data.columns[0],  # Set default value to the first column
                    style={'max-width': '250px'}
                )
            ], width = 3),
            dbc.Col(id='slider-container')
        ], style={'margin-bottom': '15px'}),
        dbc.Row([
            dbc.Col([
                html.Label("Select a Filter Feature:", style={'margin-bottom': '10px'}),
                dbc.Select(
                    id='condition-column-dropdown',
                    options=[{'label': col, 'value': col} for col in data.columns],
                    value=data.columns[0],  # Set default value to the first column
                    style={'max-width': '250px'}
                )
            ], width = 3),
            dbc.Col([
                html.Label("Select a Filter Condition:", style={'margin-bottom': '10px'}),
                dbc.Select(
                    id='condition-dropdown',
                    options=[],
                    value=None,
                    style={'margin-bottom': '10px'}
                ),
                html.Div(id='filter-input-container')
            ], width=6),
            dbc.Col([
                dbc.Button('Apply Filter', id='apply-button', n_clicks=0, style={'margin-right': '10px'}),
                dbc.Button('Clear All Filters', id='clear-button', n_clicks=0, style={'margin-left': '10px'})
            ], width=3, style={
                'display': 'flex',
                'justify-content': 'flex-end',  # Centers the button horizontally
                'align-items': 'center',  # Centers the button vertically
            })
        ]),
        dbc.Row([
            dbc.Col([
                dcc.Graph(figure={}, id='col_histogram')
            ], width=6)
            ,dbc.Col([
                dcc.Graph(figure={}, id='col_boxplot')
            ], width=6)
        ])
    ], style={'padding': '20px'})
])

In [80]:
# Show slider/checklist
@callback(
    Output('slider-container', 'children'),
    Input('column-dropdown', 'value')
)

def update_slider(col_chosen):
    if col_chosen in no_categorical:
        return [
            html.Label("Filter the Displayed Feature:", style={'margin-bottom': '10px'}),
            dcc.RangeSlider(
                id='value-input',
                min=data[col_chosen].min(),
                max=data[col_chosen].max(),
                step=1,
                tooltip={"always_visible": False, "placement": "bottom"},
                marks={i: str(i) for i in range(int(data[col_chosen].min()), int(data[col_chosen].max()) + 1, 10)},
                value=[data[col_chosen].min(), data[col_chosen].max()]
            )
        ]
    else:
        unique_values = data[col_chosen].unique().tolist()

        return [
            html.Label("Filter the Displayed Feature:", style={'margin-bottom': '10px'}),
            dcc.Checklist(
                id='value-input',
                options=[{'label': html.Label(val, style={'margin-right': '15px', 'margin-left': '5px'}), 'value': val} for val in unique_values],
                value=unique_values,
                inline=True
            )
        ]

In [81]:
# Condition Interaction
@callback(
    Output('condition-dropdown', 'options'),
    Output('condition-dropdown', 'value'),
    Input('condition-column-dropdown', 'value')
)

def update_condition_dropdown(col_chosen):
    if col_chosen in no_categorical:
        conditions = ['greater than', 'less than', 'equal to', 'greater than or equal to', 'less than or equal to']
    
    else:
        conditions = ['is', 'is not']

    options = [{'label': cond, 'value': cond} for cond in conditions]

    return options, conditions[0] if conditions else None

In [82]:
@callback(
    Output('filter-input-container', 'children'),
    Input('condition-column-dropdown', 'value')
)


def update_filter_input(col_chosen):
    if col_chosen in no_categorical:
        # If the selected column is continuous, show the range slider
        return [
            html.Label("Filter the Displayed Feature:", style={'margin-bottom': '10px'}),
            dbc.Input(id='filter-input', type='text', value='', placeholder="Enter value", style={'margin-bottom': '15px'})
        ]
    else:
        # If the selected column is categorical, show the dropdown
        unique_values = data[col_chosen].unique().tolist()
        return [
            html.Label("Filter the Displayed Feature:", style={'margin-bottom': '10px'}),
            dbc.Select(
                id='filter-input',
                options=[{'label': val, 'value': val} for val in unique_values],
                value=unique_values[0] if unique_values else None,  # Default to the first value if exists
                style={'margin-bottom': '15px'}
            )
        ]

In [83]:
# Graphs Interaction
@callback(
    [Output('col_histogram', 'figure'),
    Output('col_boxplot', 'figure'),
    Output('apply-button', 'n_clicks'),
    Output('clear-button', 'n_clicks')],
    [Input('column-dropdown', 'value'),
    Input('value-input', 'value'),
    Input('condition-column-dropdown', 'value'),
    Input('condition-dropdown', 'value'),
    Input('filter-input', 'value'),
    Input('apply-button', 'n_clicks'),
    Input('clear-button', 'n_clicks')]
)

def update_graph(col_chosen, value_input, col_condition, condition, value, n_clicks, n_clicks_clear):
        
    if n_clicks_clear > 0:
        # Determine filtering logic for the input column
        if col_chosen in no_categorical:
            filtered_df = data[(data[col_chosen] >= value_input[0]) & (data[col_chosen] <= value_input[1])]
        elif col_chosen in non_metric_features:
            filtered_df = data[data[col_chosen].isin(value_input)]
        else:
            filtered_df = data
            
    else:
        # Determine filtering logic for the input column
        if col_chosen in no_categorical:
            filtered_df = data[(data[col_chosen] >= value_input[0]) & (data[col_chosen] <= value_input[1])]
        elif col_chosen in non_metric_features:
            filtered_df = data[data[col_chosen].isin(value_input)]
        else:
            filtered_df = data

        # Determine the filtering logic for the filtering column
        if value and n_clicks > 0:
            if condition == 'is':
                filtered_df = filtered_df[filtered_df[col_condition].astype('str') == str(value)]
            
            elif condition == 'is not':
                filtered_df = filtered_df[filtered_df[col_condition].stype('str') != str(value)]

            elif condition == 'greater than':
                filtered_df = filtered_df[filtered_df[col_condition] > float(value)]

            elif condition == 'less than':
                filtered_df = filtered_df[filtered_df[col_condition] < float(value)]

            elif condition == 'equal to':
                filtered_df = filtered_df[filtered_df[col_condition] == float(value)]

            elif condition == 'greater than or equal to':
                filtered_df = filtered_df[filtered_df[col_condition] >= float(value)]

            elif condition == 'less than or equal to':
                filtered_df = filtered_df[filtered_df[col_condition] <= float(value)]

    fig_hist = px.histogram(filtered_df, x=col_chosen)
    if col_chosen in no_categorical:
        fig_box = px.box(filtered_df, y=col_chosen)  # TODO: change color
        fig_box.update_traces(marker=dict(color="#E145B4"))
    else:
        # Create an empty figure with a transparent rectangle
        fig_box = go.Figure()

        fig_box.update_layout(
            shapes=[
                go.layout.Shape(
                    type="rect",
                    x0=0, x1=1, y0=0, y1=1,
                    xref="paper", yref="paper",
                    line=dict(color="white"),  # This draws a white rectangle
                    fillcolor="white"  # Transparent rectangle
                )
            ], xaxis=dict(showline=False, showgrid=False, zeroline=False, showticklabels=False),  # Hide x-axis
            yaxis=dict(showline=False, showgrid=False, zeroline=False, showticklabels=False)   # Hide y-axis
        )
    
    return fig_hist, fig_box, 0, 0

## Pairplot

In [84]:
pairplot = html.Div([
    dbc.Container([
        dbc.Row([
            html.Div('Pairplot Exploration', className="text-primary text-center fs-3", style={'margin-bottom': '15px'})
        ]),
        dbc.Row([
            dbc.Col([
                html.Label("Select a Feature:", style={'margin-bottom': '10px'}),
                dbc.Select(
                    id='pairplot-feature-1',  # Ensure this ID is correctly defined
                    options=[{'label': col, 'value': col} for col in data.columns],
                    value=data.columns[0],  # Set default value to the first column
                    style={'max-width': '250px'}
                )
            ], width=3),
            dbc.Col([
                html.Label("Select another Feature:", style={'margin-bottom': '10px'}),
                dbc.Select(
                    id='pairplot-feature-2',  # Ensure this ID is correctly defined
                    options=[{'label': col, 'value': col} for col in data.columns],
                    value=data.columns[0],  # Set default value to the first column
                    style={'max-width': '250px'}
                )
            ], width=3)
        ], style={'margin-bottom': '15px'}),
        # Filter Section
        dbc.Row([
            dbc.Col([
                html.Label("Select a Filter Feature:", style={'margin-bottom': '10px'}),
                dbc.Select(
                    id='filter-column-dropdown',
                    options=[{'label': col, 'value': col} for col in data.columns],
                    value=data.columns[0],  # Set default value to the first column
                    style={'max-width': '250px'}
                )
            ], width = 3),
            dbc.Col([
                html.Label("Select a Filter Condition:", style={'margin-bottom': '10px'}),
                dbc.Select(
                    id='filter-condition-dropdown',
                    options=[],
                    value=None,  # Default filter condition
                    style={'margin-bottom': '10px'}
                ),
                html.Div(id='pairplot-input-container')
            ], width=6),
            dbc.Col([
                dbc.Button('Apply Filter', id='apply-filter-button', n_clicks=0, style={'margin-right': '10px'}),
                dbc.Button('Clear Filter', id='clear-filter-button', n_clicks=0, style={'margin-left': '10px'})
            ], width=3,  style={
                'display': 'flex',
                'justify-content': 'flex-end',  # Centers the button horizontally
                'align-items': 'center',  # Centers the button vertically
            })
        ], style={'margin-bottom': '15px'}),
        dbc.Row([
            dbc.Col([
                dcc.Graph(figure={}, id='graph-pairplot')
            ], style={
                'display': 'flex',
                'justify-content': 'center',  # Centers the button horizontally
                'align-items': 'center',  # Centers the button vertically
            })
        ])
    ], style={'padding': '20px'})
])

In [85]:
# Condition Interaction
@callback(
    Output('filter-condition-dropdown', 'options'),
    Output('filter-condition-dropdown', 'value'),
    Input('filter-column-dropdown', 'value')
)

def update_condition_dropdown(col_chosen):
    if col_chosen in no_categorical:
        conditions = ['greater than', 'less than', 'equal to', 'greater than or equal to', 'less than or equal to']
    
    else:
        conditions = ['is', 'is not']

    options = [{'label': cond, 'value': cond} for cond in conditions]

    return options, conditions[0] if conditions else None

In [86]:
@callback(
    Output('pairplot-input-container', 'children'),
    Input('filter-column-dropdown', 'value')
)


def update_filter_input(col_chosen):
    if col_chosen in no_categorical:
        # If the selected column is continuous, show the range slider
        return [
            html.Label("Filter the Displayed Feature:", style={'margin-bottom': '10px'}),
            dbc.Input(id='pairplot-filter-input', type='text', value='', placeholder="Enter value", style={'margin-bottom': '15px'})
        ]
    else:
        # If the selected column is categorical, show the dropdown
        unique_values = data[col_chosen].unique().tolist()
        return [
            html.Label("Filter the Displayed Feature:", style={'margin-bottom': '10px'}),
            dbc.Select(
                id='pairplot-filter-input',
                options=[{'label': val, 'value': val} for val in unique_values],
                value=unique_values[0] if unique_values else None,  # Default to the first value if exists
                style={'margin-bottom': '15px'}
            )
        ]

In [87]:
# Callback to handle filtering and updating the pairplot
@callback(
    Output('graph-pairplot', 'figure'),
    Output('apply-filter-button', 'n_clicks'),
    Output('clear-filter-button', 'n_clicks'),
    Output('pairplot-filter-input', 'value'),
    Input('pairplot-feature-1', 'value'),
    Input('pairplot-feature-2', 'value'),
    Input('filter-column-dropdown', 'value'),
    Input('filter-condition-dropdown', 'value'),
    Input('pairplot-filter-input', 'value'),
    Input('apply-filter-button', 'n_clicks'),
    Input('clear-filter-button', 'n_clicks')
)

def update_pairplot(feature_1, feature_2, filter_column, filter_condition, filter_value, apply_clicks, clear_clicks):
    # Apply the filter to the DataFrame
    filtered_df = data.copy()
    
    # If the 'Clear Filter' button was clicked, reset filter
    if clear_clicks > 0:
        filter_value = ''
    
    elif filter_value and filter_column and apply_clicks > 0:
        if filter_condition == 'is':
            filtered_df = filtered_df[filtered_df[filter_column].astype('str') == str(filter_value)]
        elif filter_condition == 'is not':
            filtered_df = filtered_df[filtered_df[filter_column].astype('str') != str(filter_value)]
        elif filter_condition == 'greater than':
            filtered_df = filtered_df[filtered_df[filter_column] > float(filter_value)]

        elif filter_condition == 'less than':
            filtered_df = filtered_df[filtered_df[filter_column] < float(filter_value)]

        elif filter_condition == 'equal to':
            filtered_df = filtered_df[filtered_df[filter_column] == float(filter_value)]

        elif filter_condition == 'greater than or equal to':
            filtered_df = filtered_df[filtered_df[filter_column] >= float(filter_value)]

        elif filter_condition == 'less than or equal to':
            filtered_df = filtered_df[filtered_df[filter_column] <= float(filter_value)]

    # Create the pairplot based on the filtered data
    pairplot_figure = px.scatter_matrix(filtered_df, dimensions=[feature_1, feature_2])

    return pairplot_figure, 0, 0, filter_value  # Return the updated figure and reset the filter input

## Clustering

### Spending Diversity

In [88]:
spending = html.Div([
    dbc.Container([
        dbc.Row([
            html.H1('Spending Diversity Cluster Exploration', className="text-center", style={'margin-bottom': '15px'})
        ]),
        # Filter Section
        dbc.Row([
            dbc.Col([
                html.Label("Select a Filter Feature:", className='text-primary', style={'margin-bottom': '10px'}),
                dbc.Select(
                    id='spending-filter-column',
                    options=[{'label': col, 'value': col} for col in spending_diversity_df.columns if col != 'labels'],
                    value=spending_diversity_df.columns[0],  # Set default value to the first column
                    style={'max-width': '250px'}
                )
            ], width = 3),
            dbc.Col([
                html.Label("Select a Filter Condition:", className='text-primary', style={'margin-bottom': '10px'}),
                dbc.Select(
                    id='spending-filter-condition',
                    options=[],
                    value=None,  # Default filter condition
                    style={'margin-bottom': '10px'}
                ),
                html.Div(id='spending-filter-input-container')
                # dbc.Input(id='spending-filter-input', type='text', value='', placeholder="Enter value", style={'margin-bottom': '15px'})
            ], width=6),
            dbc.Col([
                dbc.Button('Apply Filter', id='spending-apply-filter-button', n_clicks=0, style={'margin-right': '10px'}),
                dbc.Button('Clear Filter', id='spending-clear-filter-button', n_clicks=0, style={'margin-left': '10px'})
            ], width=3,  style={
                'display': 'flex',
                'justify-content': 'flex-end',  # Centers the button horizontally
                'align-items': 'center',  # Centers the button vertically
            })
        ], style={'margin-bottom': '15px'}),
        dbc.Row([
            html.H2("Cluster Visualization", className='text-secondary', style={'margin-bottom': '20px'}),
            dbc.Col([
                dcc.Graph(figure={}, id='graph-spending-diversity-pairplot')
            ], style={
                'display': 'flex',
                'justify-content': 'center',
                'align-items': 'center',
            })
        ], style={'margin-bottom': '15px'}),
        dbc.Row(style={'height': '15px'}),
        dbc.Row([
            html.H2("Cluster Profiling Heatmap", className='text-secondary', style={'margin-bottom': '20px'}),
            dbc.Col([
                dcc.Graph(figure={}, id='profiling-spending-heatmap')
            ], style={
                'display': 'flex',
                'justify-content': 'left',
                'align-items': 'center',
            }, width=7),
            dbc.Col([
                dcc.Markdown('''
                    **Cluster 0 – Adventurous High-Spenders:** These customers are explorers, frequently trying new cuisines
                    and vendors. They also buy many products and spend a significant amount of money. They value
                    variety and have high purchasing power, likely enjoying discovering new options and experiences.
                             
                    **Cluster 1 – Loyal High-Spenders:** These customers spend a lot of money and buy many products but
                    prefer sticking to familiar cuisines and vendors. They exhibit loyalty to a select range of offerings while
                    demonstrating significant spending capacity.
                             
                    **Cluster 2 – Low-Spending Minimalists:** These customers have the lowest spending, try the fewest
                    cuisines and vendors, and purchase the least products. They are cost-conscious and not very
                    exploratory, possibly focusing on essentials or sticking to a routine.
                             
                    **Cluster 3 – Exploratory Budget-Conscious:** These customers enjoy trying different cuisines and
                    vendors but do not purchase many products or spend much money. They prioritize variety and
                    experiences but are budget-conscious or limit their purchases.
                ''')
            ], width=5)
        ], style={'margin-bottom': '15px'}),
        dbc.Row([
            html.H2("Cluster Profiling by Category", className='text-secondary', style={'margin-bottom': '20px'}),
            html.Label("Select a Profiling Feature:", className='text-primary', style={'margin-bottom': '10px'}),
            dbc.Select(
                id='spending-profiling-select',
                options=[{'label': col, 'value': col} for col in non_metric_features[:5]],
                style={'margin-bottom': '10px'}
            ),
            dash_table.DataTable(data=pd.DataFrame().to_dict('records'), id='spending-profiling-table')
        ])
    ], style={'padding': '20px'})
])

In [89]:
@callback(
    Output('spending-profiling-table', 'data'),
    Input('spending-profiling-select', 'value')
)

def update_profiling_table(feature):
    if feature is not None:
        feature = next((key for key, value in mapping_dict.items() if value == feature), None)
        
        df = spending_diversity_profiling.groupby('labels')[[col for col in spending_diversity_profiling.columns if col.startswith(feature)]].mean().round(2).reset_index()
    
        return df.to_dict('records')

In [90]:
# Condition Interaction
@callback(
    Output('spending-filter-condition', 'options'),
    Output('spending-filter-condition', 'value'),
    Input('spending-filter-column', 'value')
)

def update_condition_dropdown(col_chosen):
    if col_chosen in no_categorical:
        conditions = ['greater than', 'less than', 'equal to', 'greater than or equal to', 'less than or equal to']
    
    else:
        conditions = ['is', 'is not']

    options = [{'label': cond, 'value': cond} for cond in conditions]

    return options, conditions[0] if conditions else None

In [91]:
@callback(
    Output('spending-filter-input-container', 'children'),
    Input('spending-filter-column', 'value')
)


def update_filter_input(col_chosen):
    if col_chosen in no_categorical:
        # If the selected column is continuous, show the range slider
        return [
            html.Label("Filter the Displayed Feature:", style={'margin-bottom': '10px'}),
            dbc.Input(id='spending-filter-input', type='text', value='', placeholder="Enter value", style={'margin-bottom': '15px'})
        ]
    else:
        # If the selected column is categorical, show the dropdown
        unique_values = spending_diversity_df[col_chosen].unique().tolist()
        return [
            html.Label("Filter the Displayed Feature:", style={'margin-bottom': '10px'}),
            dbc.Select(
                id='spending-filter-input',
                options=[{'label': val, 'value': val} for val in unique_values],
                value=unique_values[0] if unique_values else None,  # Default to the first value if exists
                style={'margin-bottom': '15px'}
            )
        ]

In [92]:
@callback(
    Output('graph-spending-diversity-pairplot', 'figure'),
    Output('profiling-spending-heatmap', 'figure'),
    Output('spending-apply-filter-button', 'n_clicks'),
    Output('spending-clear-filter-button', 'n_clicks'),
    Output('spending-filter-input', 'value'),
    Input('spending-filter-column', 'value'),
    Input('spending-filter-condition', 'value'),
    Input('spending-filter-input', 'value'),
    Input('spending-apply-filter-button', 'n_clicks'),
    Input('spending-clear-filter-button', 'n_clicks')
)

def update_spending_graph(filter_column, filter_condition, filter_value, apply_clicks, clear_clicks):
    # Apply the filter to the DataFrame
    filtered_df = spending_diversity_df.copy()
    
    # If the 'Clear Filter' button was clicked, reset filter
    if clear_clicks > 0:
        filter_value = ''
    
    elif filter_value and filter_column and apply_clicks > 0:
        if filter_condition == 'is':
            filtered_df = filtered_df[filtered_df[filter_column].astype('str') == str(filter_value)]
        elif filter_condition == 'is not':
            filtered_df = filtered_df[filtered_df[filter_column].astype('str') != str(filter_value)]
        elif filter_condition == 'greater than':
            filtered_df = filtered_df[filtered_df[filter_column] > float(filter_value)]

        elif filter_condition == 'less than':
            filtered_df = filtered_df[filtered_df[filter_column] < float(filter_value)]

        elif filter_condition == 'equal to':
            filtered_df = filtered_df[filtered_df[filter_column] == float(filter_value)]

        elif filter_condition == 'greater than or equal to':
            filtered_df = filtered_df[filtered_df[filter_column] >= float(filter_value)]

        elif filter_condition == 'less than or equal to':
            filtered_df = filtered_df[filtered_df[filter_column] <= float(filter_value)]

    filtered_df['labels'] = filtered_df['labels'].astype(str)
    
    combinations = list(itertools.combinations(spending_diversity_features, 2))
    n_combinations = len(combinations)

    # Define grid layout
    n_cols = 3
    n_rows = (n_combinations + n_cols - 1) // n_cols

    used_labels = set()

    # Create subplots
    fig = make_subplots(
        rows=n_rows, cols=n_cols,
        subplot_titles=[f"{x} vs {y}" for x, y in combinations]
    )

    # Add scatter plots using Plotly Express
    row = col = 1
    for feature_x, feature_y in combinations:
        scatter_fig = px.scatter(
            filtered_df,
            x=feature_x,
            y=feature_y,
            color='labels',
            opacity=0.5,
            color_discrete_sequence=px.colors.qualitative.Vivid
        )

        # Add traces from Plotly Express figure to the subplot
        for trace in scatter_fig.data:
            if trace.name in used_labels:
                trace.showlegend = False
            else:
                used_labels.add(trace.name)

            fig.add_trace(trace, row=row, col=col)
        
        # Update row and column indices
        col += 1
        if col > n_cols:
            col = 1
            row += 1

    # Update layout
    fig.update_layout(
        height=n_rows * 400,
        width=1000,
        showlegend=True,
        legend_title_text='Clusters'
    )

    hm = px.imshow(
        filtered_df[spending_diversity_features + ['labels']].groupby('labels').mean().T,
        text_auto=".2f",
        color_continuous_scale="rdylgn",
        labels={"x": "Cluster Labels", "y": "Features", "color": "Mean Value"}
    )

    hm.update_layout(
        xaxis_title="Cluster Labels",
        yaxis_title="Features",
    )

    return fig, hm, 0, 0, filter_value

## Page Navigation

In [93]:
@callback(
    [Output('page-content', 'children'),
    Output('home-button', 'active'),
    Output('single-feature-button', 'active'),
    Output('pairplot-button', 'active'),
    Output('spending-diversity-button', 'active')],
    [Input('url', 'pathname')]
)

def display_page(pathname):
    if pathname == '/single-feature':
        return single_feature, False, True, False, False
    elif pathname == '/pairplot':
        return pairplot, False, False, True, False
    elif pathname == '/spending':
        return spending, False, False, False, True
    else:
        return home_layout, True, False, False, False

# Dashboard

In [94]:
# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8052)

Promotion
last_promo
